1.18 面向对象

1.18.1 继承

In [1]:
# 简单的继承
class Animal:
    def __init__(self, name):
        self.name = name

    def yell(self):
        print("%s: %s" % (self.name, "..."))


class Cat(Animal):
    def __init__(self, name):
        Animal.__init__(self, name)

a, b = Animal("a"), Cat("b")
a.yell(), b.yell()
print(a.__class__, b.__class__)

a: ...
b: ...
<class '__main__.Animal'> <class '__main__.Cat'>


In [6]:
# 其他调用父类函数的写法
# 直接用类名调用父类方法在使用单继承的时候没问题，但是如果使用多继承，会涉及到查找顺序（MRO）、重复调用（钻石继承）等种种问题
# 使用 super() 可解决多继承问题，后文会详细介绍
class Animal:
    def __init__(self, name):
        self.name = name

    def yell(self):
        print("%s: %s" % (self.name, "..."))


class Cat(Animal):
    def __init__(self, name):
        # 第一种 super 的写法，括号内参数为（子类名，self）
        super(Cat, self).__init__(name)
        
    def yell(self):
        # 第二种 super 的写法，不需要参数也可以（在类中才能这样写）
        super().yell()

a, b = Animal("a"), Cat("b")
a.yell(), b.yell()
print(a.__class__, b.__class__)

a: ...
b: ...
<class '__main__.Animal'> <class '__main__.Cat'>


In [7]:
# 方法重写
class Animal:
    def __init__(self, name):
        self.name = name

    def yell(self):
        print("%s: %s" % (self.name, "..."))


class Cat(Animal):
    def __init__(self, name):
        Animal.__init__(self, name)

    def yell(self):
        print("%s: %s" % (self.name, "Meow"))

a, b = Animal("a"), Cat("b")
a.yell(), b.yell()
print(a.__class__, b.__class__)

# 也可以通过 super 来调用父类的 yell()
super(Cat, b).yell()

a: ...
b: Meow
<class '__main__.Animal'> <class '__main__.Cat'>
b: ...


In [15]:
# 多继承
# 因为多继承传递的参数不一致，所以使用不定参数（*args, **kwargs）

# 动物类
class Animal:
    def __init__(self, name, *args, **kwargs):
        self.name = name

    def yell(self):
        print("%s: %s" % (self.name, "..."))


# 猫类
class Cat(Animal):
    def __init__(self, name, age, *args, **kwargs):
        super().__init__(name, age)
        self.age = age

    def yell(self):
        print("%s: %s" % (self.name, "Meow"))

    def get_age(self):
        print("I am %d years old" % self.age)


# 战士类
class Warrior:
    def __init__(self, name, weapon, *args, **kwargs):
        self.name = name
        self.weapon = weapon

    def yell(self):
        print("%s: %s" % (self.name, "!!!"))

    def get_weapon(self):
        print("My weapon is %s" % self.weapon)


# 同时继承 Cat 与 Warrior
class CatWarrior(Cat, Warrior):
    def __init__(self, name, age, weapon):
        self.name = name
        self.age = age
        self.weapon = weapon
        super().__init__(name, age, weapon)
        # 在此处等价于
        # Cat.__init__(self, name, age)
        # Warrior.__init__(self, name, weapon)


instance = CatWarrior('Mario', 1, 'Sword')
instance.yell()    # Cat 与 Warrior 都有 yell() 方法时，先调用继承时处在括号最左边的类的方法
instance.get_age()   # 继承自 Cat
instance.get_weapon()    # 继承自 Warrior
# 指定调用 Warrior 的 yell() 方法
Warrior.yell(instance)

Mario: Meow
I am 1 years old
My weapon is Sword
Mario: !!!


封装、继承、多态
（一段很长的例子）

In [1]:
# 导入必要的库
import copy      # 用于深拷贝
import math     # 用于获得 float 变量的绝对值

In [2]:
class Vector(object):
    def __init__(self, data: list):
        """
        Default initializer.
        :param data: external data
        """
        # detection for invalid parameters
        if type(data) is not list:
            raise ValueError("type 'list' is expected")
        # detection for invalid elements
        for val in data:
            if type(val) not in [Vector, int, float, complex]:
                raise ValueError("data type not understood")
        # create a duplicate for the original data
        self.data = copy.deepcopy(data)

    def __len__(self) -> int:
        """
        Override of __len__.
        :return: length of the vector
        """
        return len(self.data)

    def __str__(self) -> str:
        """
        Override of __str__.
        :return: content for print()
        """
        return self.data.__str__()

    def __repr__(self) -> str:
        """
        Override of __repr__.
        :return: the canonical string representation of the object
        """
        return self.data.__repr__()

    def __getitem__(self, index: 'int > 0'):
        """
        Override of __getitem__.
        :param index: index of item
        :return: value of item
        """
        return self.data[index]

    def __setitem__(self, index: 'int > 0', value):
        """
        Override of __setitem__.
        :param index: index of item
        :param value: new value of item
        :return: value of item
        """
        self.data[index] = value
        return self.data[index]

    def __iter__(self):
        """
        Override of __iter__.
        :return: object for iteration
        """
        self.it = iter(self.data)
        return self

    def __next__(self):
        """
        Override of __next__
        :return: next element
        """
        return next(self.it)

    @classmethod
    def zeros(cls, dimension: 'int > 0'):
        """
        Return a new vector of given dimension, filled with zeros.
        :param dimension: dimension of vector
        :return: a new Vector object
        """
        # detection for invalid parameters
        if dimension < 0:
            raise ValueError("negative dimensions are not allowed")
        # call initializer
        return cls([0] * dimension)

    @classmethod
    def zeros_likes(cls, other):
        """
        Return a vector of zeros with the same dimension as a given vector (or list).
        :param other: another Vector (or list)
        :return: a new Vector object
        """
        # detection for invalid parameters
        if type(other) is not list and type(other) is not Vector:
            raise ValueError("type 'list' or type 'Vector' is expected")
        # call initializer
        return cls([0] * len(other))

    def __add__(self, other):
        """
        Perform addition on two vectors.
        :param other: another Vector
        :return: a new Vector object
        """
        # detection for invalid parameters
        if type(other) is not Vector:
            raise ValueError("type 'Vector' is expected")
        elif len(self) != len(other):
            raise ValueError("operands could not be broadcast together with dimensions %d %d" % (len(self), len(other)))
        # add operation
        data = copy.deepcopy(self.data)
        for i in range(len(data)):
            data[i] += other[i]
        # return
        return Vector(data)

    def __sub__(self, other):
        """
        Perform subtraction on two vectors.
        :param other: another Vector
        :return: a new Vector object
        """
        # detection for invalid parameters
        if type(other) is not Vector:
            raise ValueError("type 'Vector' is expected")
        elif len(self) != len(other):
            raise ValueError("operands could not be broadcast together with dimensions %d %d" % (len(self), len(other)))
        # sub operation
        data = copy.deepcopy(self.data)
        for i in range(len(data)):
            data[i] -= other[i]
        # return
        return Vector(data)

    def __mul__(self, other):
        """
        Perform multiplication on two vectors.
        :param other: another Vector
        :return: a new Vector object
        """
        # detection for invalid parameters
        if type(other) is not Vector:
            raise ValueError("type 'Vector' is expected")
        elif len(self) != len(other):
            raise ValueError("operands could not be broadcast together with dimensions %d %d" % (len(self), len(other)))
        # mul operation
        data = copy.deepcopy(self.data)
        for i in range(len(data)):
            data[i] *= other[i]
        # return
        return Vector(data)

    def __matmul__(self, other):
        """
        Perform dot product on two vectors.
        :param other: another Vector
        :return: result of dot product
        """
        # detection for invalid parameters
        if type(other) is not Vector:
            raise ValueError("type 'Vector' is expected")
        elif len(self) != len(other):
            raise ValueError("operands could not be broadcast together with dimensions %d %d" % (len(self), len(other)))
        # return
        return sum(self.__mul__(other))

In [7]:
class Matrix(Vector):
    def __init__(self, data: list):
        """
        Default initializer.
        :param data: external data
        """
        # detection for invalid parameters
        if type(data) is not list:
            raise ValueError("type 'list' is expected")
        # detection for invalid elements
        self.dimension = 0
        for i in range(len(data)):
            if type(data[i]) is list:
                data[i] = Vector(data[i])
            if type(data[i]) is not Vector:
                raise ValueError("data type not understood")
            elif 0 < self.dimension != len(data[i]):
                raise ValueError("inconsistent dimensions are not allowed")
            self.dimension = len(data[i])
        # call initializer
        super(Matrix, self).__init__(data)

    def __str__(self) -> str:
        """
        Override of __str__.
        :return: content for print()
        """
        return self.data.__str__().replace("], ", "]\n ")

    def __repr__(self) -> str:
        """
        Override of __repr__.
        :return: the canonical string representation of the object
        """
        return self.data.__repr__().replace("], ", "]\n ")

    def __getitem__(self, index):
        """
        Override of __getitem__.
        :param index: index of item
        :return: value of item
        """
        return self.data[index[0]][index[1]]

    def __setitem__(self, index, value):
        """
        Override of __setitem__.
        :param index: index of item
        :param value: new value of item
        :return: value of item
        """
        self.data[index[0]][index[1]] = value
        return self.data[index[0]][index[1]]

    def shape(self):
        """
        Get the shape of matrix.
        :return: the shape of matrix
        """
        return len(self), self.dimension

    @classmethod
    def zeros(cls, shape):
        """
        Return a new matrix of given dimension, filled with zeros.
        :param shape: the shape of matrix
        :return: a new matrix object
        """
        # detection for invalid parameters
        if len(shape) != 2:
            raise ValueError("shape not understood")
        elif shape[0] < 0 or shape[1] < 0:
            raise ValueError("negative dimensions are not allowed")
        # call initializer
        return cls([Vector.zeros(shape[1]) for _ in range(shape[0])])

    @classmethod
    def zeros_likes(cls, other):
        """
        Return a matrix of zeros with the same dimension as a given matrix.
        :param other: another Matrix (or list)
        :return: a new Matrix object
        """
        # detection for invalid parameters
        if type(other) is not Matrix:
            raise ValueError("type 'Matrix' is expected")
        # call initializer
        shape = other.shape()
        return cls([Vector.zeros(shape[1]) for _ in range(shape[0])])

    @classmethod
    def diagonal(cls, dimension):
        """
        Get a diagonal matrix with given shape
        :param dimension: the shape of matrix
        :return: a diagonal matrix
        """
        # detection for invalid parameters
        if type(dimension) is not int:
            raise ValueError("type 'int' is expected")
        elif dimension < 0:
            raise ValueError("negative dimensions are not allowed")
        # call initializer
        matrix = cls.zeros(shape=(dimension, dimension))
        for i in range(dimension):
            matrix[i, i] = 1
        return matrix

    def __add__(self, other):
        """
        Perform addition on two matrices.
        :param other: another Matrix
        :return: a new Matrix object
        """
        # detection for invalid parameters
        if type(other) is not Matrix:
            raise ValueError("type 'Matrix' is expected")
        elif self.shape() != other.shape():
            raise ValueError("operands could not be broadcast together with dimensions ", self.shape(), other.shape())
        # add operation
        data, shape = copy.deepcopy(self.data), self.shape()
        for i in range(shape[0]):
            for j in range(shape[1]):
                data[i][j] += other[i, j]
        # return
        return Matrix(data)

    def __sub__(self, other):
        """
        Perform subtraction on two matrices.
        :param other: another Matrix
        :return: a new Matrix object
        """
        # detection for invalid parameters
        if type(other) is not Matrix:
            raise ValueError("type 'Matrix' is expected")
        elif self.shape() != other.shape():
            raise ValueError("operands could not be broadcast together with dimensions ", self.shape(), other.shape())
        # sub operation
        data, shape = copy.deepcopy(self.data), self.shape()
        for i in range(shape[0]):
            for j in range(shape[1]):
                data[i][j] -= other[i, j]
        # return
        return Matrix(data)

    def __mul__(self, other):
        """
        Perform multiplication on two matrices.
        :param other: another Matrix
        :return: a new Matrix object
        """
        # detection for invalid parameters
        if type(other) is not Matrix:
            raise ValueError("type 'Matrix' is expected")
        elif self.shape() != other.shape():
            raise ValueError("operands could not be broadcast together with dimensions ", self.shape(), other.shape())
        # mul operation
        data, shape = copy.deepcopy(self.data), self.shape()
        for i in range(shape[0]):
            for j in range(shape[1]):
                data[i][j] *= other[i, j]
        # return
        return Matrix(data)

    def __matmul__(self, other):
        """
        Perform dot product on two matrices.
        :param other: another Matrix
        :return: result of dot product
        """
        # detection for invalid parameters
        if type(other) is not Matrix:
            raise ValueError("type 'Matrix' is expected")
        elif self.shape()[1] != other.shape()[0]:
            raise ValueError("operands could not be broadcast together with dimensions ", self.shape(), other.shape())
        # matmul operation
        shape = (self.shape()[0], self.shape()[1], other.shape()[1])
        matrix = Matrix.zeros(shape=(shape[0], shape[2]))
        for i in range(shape[0]):
            for j in range(shape[2]):
                for k in range(shape[1]):
                    matrix[i, j] += self[i, k] * other[k, j]
        # return
        return matrix

    def transpose(self):
        """
        Get the transpose of the current matrix
        :return: the transpose of the current matrix
        """
        shape = self.shape()
        matrix = Matrix.zeros(shape=(shape[1], shape[0]))
        for i in range(shape[0]):
            for j in range(shape[1]):
                matrix[j, i] = self[i, j]
        # return
        return matrix

    def inverse(self):
        """
        Get the inverse of the current matrix.
        :return: the inverse of the current matrix
        """
        # make sure that the two dimensions are equal
        shape = self.shape()
        if shape[0] != shape[1]:
            raise ValueError()
        # create a copy for the current matrix
        matrix = Matrix(self.data)
        # perform Gaussian elimination
        n = shape[0]
        r, c = Vector.zeros(n), Vector.zeros(n)
        # eliminate
        for k in range(n):
            d = 0.0
            for y in range(k, n):
                for x in range(k, n):
                    if math.fabs(matrix[y, x]) > d:
                        d = math.fabs(matrix[y, x])
                        r[k] = y
                        c[k] = x
            # have no solution if the determinant equals zero
            if math.fabs(d) < 1e-5:
                return Matrix.zeros(shape=shape)
            if r[k] != k:
                for j in range(n):
                    matrix[k, j], matrix[r[k], j] = matrix[r[k], j], matrix[k, j]
            if c[k] != k:
                for i in range(n):
                    matrix[i, k], matrix[i, c[k]] = matrix[i, c[k]], matrix[i, k]
            # scale
            matrix[k, k] = 1.0 / matrix[k, k]
            for j in range(n):
                if j != k:
                    matrix[k, j] *= matrix[k, k]
            for i in range(n):
                if i != k:
                    for j in range(n):
                        if j != k:
                            matrix[i, j] -= matrix[i, k] * matrix[k, j]
            for i in range(n):
                if i != k:
                    matrix[i, k] *= -matrix[k, k]
        # swap
        for k in range(n-1, -1, -1):
            if c[k] != k:
                for j in range(n):
                    matrix[k, j], matrix[c[k], j] = matrix[c[k], j], matrix[k, j]
            if r[k] != k:
                for i in range(n):
                    matrix[i, k], matrix[i, r[k]] = matrix[i, r[k]], matrix[i, k]
        # return
        return matrix

In [4]:
# 测试 Vector
v1 = Vector([4, 2, 3])
v2 = Vector([9, 5, 7])

for x in v1:
    print(x, end=' ')
print()

print(v1 + v2)
print(v1 - v2)
print(v1 * v2)
print(v1 @ v2)

4 2 3 
[13, 7, 10]
[-5, -3, -4]
[36, 10, 21]
67


In [8]:
# 测试 Matrix
m1 = Matrix([[1.0000, 1.0000, 1.0000], [1.0000, 2.0000, 3.0000], [1.0000, 5.0000, 1.0000]])
m2 = Matrix.diagonal(dimension=3)

print(m1 * m2)
print(m1 @ m1)
print(m1.transpose())
print(m1.inverse())

[[1.0, 0.0, 0.0]
 [0.0, 2.0, 0.0]
 [0.0, 0.0, 1.0]]
[[3.0, 8.0, 5.0]
 [6.0, 20.0, 10.0]
 [7.0, 16.0, 17.0]]
[[1.0, 1.0, 1.0]
 [1.0, 2.0, 5.0]
 [1.0, 3.0, 1.0]]
[[1.625, -0.5, -0.125]
 [-0.25, 0.0, 0.25]
 [-0.37499999999999994, 0.49999999999999994, -0.125]]
